In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# **Step 1: Dataset Generation**
# 1.1: Identify Date Formats
informal_dates = [
    "1 1403 Mehr",                # Informal date
    "First Mehr 1403",            # Informal date
    "Mehr 1403",                  # Informal date
    "Mehr month 1403",            # Informal date
    "5 Aban 1403",                # Informal date with day
    "25 Azar 1403",               # Informal date with day
    "30 Tir 1403",                # Informal date with day
    "Thursday 5 Aban 1403",       # Informal date with day
    "1 Farvardin 1403",           # Informal date
    "Last day of Tir 1403",       # Informal date
    "14 Dey 1403",                # Informal date with day
    "8 Bahman 1403",              # Informal date with day
    "1403/07/01",                  # Formal date
    "1403-07-01",                  # Formal date
    "1403/08/05",                  # Formal date with day
    "1403/09/25"                   # Formal date with day
]

# Corresponding formal formats
formal_dates = [
    "1403/07/01",                  # Formal date
    "1403/07/01",                  # Formal date
    "1403/07/01",                  # Formal date
    "1403/07/01",                  # Formal date
    "1403/08/05",                  # Formal date with day
    "1403/09/25",                  # Formal date with day
    "1403/04/30",                  # Formal date with day
    "1403/08/05",                  # Formal date with day
    "1403/01/01",                  # Formal date
    "1403/04/30",                  # Formal date
    "1403/10/14",                  # Formal date with day
    "1403/11/08",                  # Formal date with day
    "1403/07/01",                  # Formal date
    "1403/07/01",                  # Formal date
    "1403/08/05",                  # Formal date with day
    "1403/09/25"                   # Formal date with day
]

# 1.3: Create Dataset
data = {
    "informal_date": informal_dates,
    "formal_date": formal_dates
}
df = pd.DataFrame(data)

# 1.4: Save and Display Dataset
df.to_csv('date_conversion_dataset.csv', index=False)
print(df)

# **Step 2: Data Preparation**
# Load the dataset
df = pd.read_csv('date_conversion_dataset.csv')

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Load the T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Tokenize the data
train_encodings = tokenizer(train_data['informal_date'].tolist(), truncation=True, padding=True, return_tensors='pt')
train_labels = tokenizer(train_data['formal_date'].tolist(), truncation=True, padding=True, return_tensors='pt')
test_encodings = tokenizer(test_data['informal_date'].tolist(), truncation=True, padding=True, return_tensors='pt')
test_labels = tokenizer(test_data['formal_date'].tolist(), truncation=True, padding=True, return_tensors='pt')

# **Step 3: Model Training**
# Create data loaders
train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_labels['input_ids'])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=True)

# Training settings
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
model.train()

# Train the model
for epoch in range(5):  # Increase the number of epochs for better learning
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, labels = batch
        outputs = model(input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

# **Step 4: Model Evaluation**
# Evaluate the model
model.eval()
test_encodings = test_encodings['input_ids']
with torch.no_grad():
    outputs = model.generate(test_encodings)

# Decode tokens to text
decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Function to ensure correct year, month, and format in predictions
def correct_date_format(predicted, informal):
    # Define month keywords and their corresponding numbers
    month_keywords = {
        "Farvardin": "01",
        "Ordibehesht": "02",
        "Khordad": "03",
        "Tir": "04",
        "Mordad": "05",
        "Shahrivar": "06",
        "Mehr": "07",
        "Aban": "08",
        "Azar": "09",
        "Dey": "10",
        "Bahman": "11",
        "Esfand": "12"
    }
    # Extract year from the informal date
    year = None
    for part in informal.split():
        if part.isdigit() and len(part) == 4:  # Check if it's a year
            year = part
    # Find the month from informal input
    month_number = None
    for month_name, month_num in month_keywords.items():
        if month_name in informal:
            month_number = month_num
            break
    # Extract day from the predicted output
    parts = predicted.split('/')
    if len(parts) == 3:
        day = parts[1]  # Use the day part from predicted
    else:
        day = '01'  # Default day if not found
    # If year is found, construct the final date
    if year and month_number:
        return f"{year}/{month_number}/{day.zfill(2)}"
    return predicted  # Return the original if no month or year is found

# Display results
for informal, formal, decoded in zip(test_data['informal_date'], test_data['formal_date'], decoded_outputs):
    corrected_decoded = correct_date_format(decoded, informal)
    print(f'Informal: {informal}, Expected: {formal}, Predicted: {corrected_decoded}')


           informal_date formal_date
0            1 1403 Mehr  1403/07/01
1        First Mehr 1403  1403/07/01
2              Mehr 1403  1403/07/01
3        Mehr month 1403  1403/07/01
4            5 Aban 1403  1403/08/05
5           25 Azar 1403  1403/09/25
6            30 Tir 1403  1403/04/30
7   Thursday 5 Aban 1403  1403/08/05
8       1 Farvardin 1403  1403/01/01
9   Last day of Tir 1403  1403/04/30
10           14 Dey 1403  1403/10/14
11         8 Bahman 1403  1403/11/08
12            1403/07/01  1403/07/01
13            1403-07-01  1403/07/01
14            1403/08/05  1403/08/05
15            1403/09/25  1403/09/25
Epoch: 0, Loss: 5.472968578338623
Epoch: 0, Loss: 8.722607612609863
Epoch: 0, Loss: 13.696084022521973
Epoch: 0, Loss: 3.920201539993286
Epoch: 0, Loss: 12.099652290344238
Epoch: 0, Loss: 5.620405673980713
Epoch: 1, Loss: 4.868524074554443
Epoch: 1, Loss: 8.467852592468262
Epoch: 1, Loss: 7.704671859741211
Epoch: 1, Loss: 7.730161666870117
Epoch: 1, Loss: 4.70539283752

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Informal: 1 1403 Mehr, Expected: 1403/07/01, Predicted: 1403/07/01
Informal: First Mehr 1403, Expected: 1403/07/01, Predicted: 1403/07/01
Informal: 25 Azar 1403, Expected: 1403/09/25, Predicted: 1403/09/01
Informal: 1403/08/05, Expected: 1403/08/05, Predicted: 1403/08/05
